# Parse Input

In [1]:
with open("day16_input.txt") as f:
    puzzle_input = f.read().split("\n\n")

In [2]:
def parse_tickets(ticket_list):
    tickets = ticket_list.split("\n")[1:]
    tickets = [ticket.split(",") for ticket in tickets]
    tickets = [[int(x) for x in ticket] for ticket in tickets]
    return tickets

In [3]:
valid_fields = puzzle_input[0].split("\n")
my_ticket = parse_tickets(puzzle_input[1])[0]
other_tickets = parse_tickets(puzzle_input[2])

In [4]:
def clean_fields(fields):
    fields_dict = {}
    
    for field in fields:
        field_name, vals = field.split(": ")
        ranges = [[int(x) for x in rng.split("-")] for rng in vals.split(" or ")]

        fields_dict[field_name] = ranges
    return fields_dict

In [5]:
valid_fields = clean_fields(valid_fields)

# Part 1

In [6]:
def get_overall_ranges(valid_field_values):
    
    rng1_min = min([rng1[0] for rng1,rng2 in valid_field_values])
    rng1_max = max([rng1[1] for rng1,rng2 in valid_field_values])
    
    rng2_min = min([rng2[0] for rng1,rng2 in valid_field_values])
    rng2_max = max([rng2[1] for rng1,rng2 in valid_field_values])

    # If they overlap - I could have hard coded this but I didn't
    if rng2_min < rng1_max:
        return [(rng1_min, rng2_max)]
    else:
        return [(rng1_min, rng1_max), (rng2_min, rng2_max)]

In [7]:
overall_valid_range = get_overall_ranges(valid_fields.values())

In [8]:
def check_valid_num(num, rng):
    lower,upper = rng[0]
    # we know there is only one range - this I did hard code :(
    return (num >= lower) & (num <= upper)

In [9]:
def check_valid_tickets(tickets, rng):
    
    invalid_nums = []
    num_tickets = 0
    
    for ticket in tickets:
        invalid_numbers = [x for x in ticket if not check_valid_num(x,rng)]
        if len(invalid_numbers) > 0:
            invalid_nums.extend(invalid_numbers)
            num_tickets += 1
    return sum(invalid_nums), num_tickets

In [10]:
check_valid_tickets(other_tickets, overall_valid_range)

(27898, 50)

# Part 2

In [11]:
def num_in_ranges(num, rng1, rng2):
    
    num_in_rng1 = rng1[0] <= num <= rng1[1]
    num_in_rng2 = rng2[0] <= num <= rng2[1]
    
    return num_in_rng1 | num_in_rng2

In [12]:
from collections import defaultdict
import numpy as np

In [13]:
def map_fields_to_cols(potential_fields):
    
    actual_fields = {}
    n = 1
    used = []
    
    while len(potential_fields) > 0:        
        # Assumption that one column can only be assigned to one field
        # Find the one. This is sketchy.
        field, columns = [(k,v) for (k,v) in potential_fields.items() if len(v) == n][0]
        
        # Assumption we should be narrowing to 1
        confirmed_column = [x for x in columns if x not in used][0]
        
        # Add to actual fields. 
        actual_fields[field] = confirmed_column
                
        # pop this key from the dictionary
        potential_fields.pop(field)
        
        n += 1
        used.append(confirmed_column)
        
    return actual_fields 

In [16]:
def part2(valid_fields, other_tickets, my_ticket):
    
    # Get the list of tickets that are actually valid
    valid_tickets = [ticket for ticket in other_tickets if all([check_valid_num(num, overall_valid_range) for num in ticket])]
    
    potential_fields = {key: [] for key in valid_fields}
    
    # Find all the 'columns' that could work for each field
    for field_name, rngs in valid_fields.items():
        rng1, rng2 = rngs
        
        # get a list of all ticket locs            
        potential_locs = [column for column in range(len(valid_fields)) if all([num_in_ranges(x, rng1, rng2) for x in [ticket[column] for ticket in valid_tickets]])]        
        potential_fields[field_name] = potential_locs
    
    # Process of elimination: one col will only be valid for one field, then cross them off from there
    actual_fields = map_fields_to_cols(potential_fields)
    
    # Finally, get the departure values and multiple them together
    departure_cols = [field[1] for field in actual_fields.items() if "departure" in field[0]]
    departure_values = [value for (i, value) in enumerate(my_ticket) if i in departure_cols]
         
    return np.prod(departure_values)

In [17]:
part2(valid_fields, other_tickets, my_ticket)

2766491048287